In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)

In [3]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
import operator

class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

# Creating Checkpointer for Persistence

In [4]:
from langgraph.checkpoint.sqlite import SqliteSaver

# if we refresh the code, the db will be lost, beacause is in memory
memory = SqliteSaver.from_conn_string(":memory:")

In [5]:
from langchain_core.messages import ToolMessage, SystemMessage
from langgraph.graph import StateGraph, END

class Agent:
  def __init__(self, model, tools, checkpointer, system=""):
    self.system = system  # saving system message as attribute
    graph = StateGraph(AgentState)
    graph.add_node("llm", self.call_openai)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
      "llm",
      self.exists_action,
      {True: "action", False: END}  # depending on the function's return
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile(checkpointer=checkpointer)
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_openai(self, state: AgentState):
    messages = state['messages']
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    return {"messages": [message]}
  
  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling: {t}")
      if not t['name'] in self.tools:  # Check for bad tool name from LLM
        print("\n ....bad tool name....")
        result = "bad tool name, retry"  # Instruct LLM to retry if bad
      else:
        result = self.tools[t['name']].invoke(t['args'])
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
      print("Back to the model!")
      return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [8]:
agent = Agent(llm, [tool], system=prompt, checkpointer=memory)

# Streaming

In [9]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="What is the weather in Campinas, SP?")]

### Thread Config

Keep track of different threads inside the persistent checkpointer<br>
Allows having multiple conversations at the same time<br>
Important for production applications where you generally have many users


In [10]:

thread_config = {"configurable": {"thread_id": "1"}}

Calling agent with `.stream()` instead of `.involke()`

In [11]:
for event in agent.graph.stream({"messages": messages}, thread_config):
  for v in event.values():
    print(v["messages"])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1OscbrpoFKaEzKzZN45flD8c', 'function': {'arguments': '{"query":"current weather in Campinas, SP"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 153, 'total_tokens': 176}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b72d04ae-58b3-45f9-97d1-1a0732644ea8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Campinas, SP'}, 'id': 'call_1OscbrpoFKaEzKzZN45flD8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 23, 'total_tokens': 176})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Campinas, SP'}, 'id': 'call_1OscbrpoFKaEzKzZN45flD8c', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi

In [12]:
messages = [HumanMessage(content="What about Valinhos?")]

thread = {"configurable": {"thread_id": "1"}}

for event in agent.graph.stream({"messages": messages}, thread):
  for v in event.values():
    print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dPmG3wQzMvtouzvxPmOhj6Zl', 'function': {'arguments': '{"query":"current weather in Valinhos, SP"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 845, 'total_tokens': 869}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dbda6c82-4837-49fe-bcfd-24fcfcf585bc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Valinhos, SP'}, 'id': 'call_dPmG3wQzMvtouzvxPmOhj6Zl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 845, 'output_tokens': 24, 'total_tokens': 869})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Valinhos, SP'}, 'id': 'call_dPmG3wQzMvtouzvxPmOhj6Zl', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\

In [14]:
%pip install aiosqlite

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
agent = Agent(llm, [tool], system=prompt, checkpointer=memory)

In [16]:
messages = [HumanMessage(content="What is the weather in Campinas?")]
thread = {"configurable": {"thread_id": "2"}}

async for event in agent.graph.astream_events({"messages": messages}, thread, version="v1"):
  kind = event["event"]
  if kind == "on_chat_model_stream":
    content = event["data"]["chunk"].content
    if content:
      # Empty content in the context of OpenAI means
      # that the model is asking for a tool to be invoked.
      # So we only print non-empty content
      print(content, end="|")

c:\Users\toa3ca\Desktop\Projects\langgraph101\.venv\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Campinas'}, 'id': 'call_hvJvwooA1JzJmbSdPU5U2Ahp', 'type': 'tool_call'}
Back to the model!
The| current| weather| in| Campinas|,| Brazil| is| as| follows|:

|-| **|Temperature|**|:| |16|.|1|°C| (|60|.|9|°F|)
|-| **|Condition|**|:| Part|ly| Cloud|y|
|-| **|Wind|**|:| |13|.|0| mph| (|20|.|9| k|ph|)| from| the| southeast|
|-| **|Humidity|**|:| |79|%
|-| **|Pressure|**|:| |102|3| mb|
|-| **|Visibility|**|:| |10| km|

|For| more| detailed| information|,| you| can| check| [|Weather|API|](|https|://|www|.weather|api|.com|/)| or| [|Time| and| Date|](|https|://|www|.time|and|date|.com|/weather|/b|razil|/c|amp|inas|/ext|).|